# Current Covid API Data

In [1]:
import pandas as pd

In [7]:
df_curr = pd.read_csv('https://api.covidtracking.com/v1/states/current.csv')

In [23]:
cols = ['state', 'positive']

In [26]:
df_curr[cols].shape

(56, 2)

In [28]:
df_curr[df['positive'] == 0]

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
3,20200824,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,c878baea61754904c3c233d78e523723e2f4a8e1,0,0,0,0,0,NaN


# Historical Covid API Data